# TPC4:  Ficheiros CSV com listas e funções de agregação

(publicado em **2023.03.06**)

Cria um programa em Python que implementa um conversor de um ficheiro CSV (Comma separated values) para o formato JSON.
Para se poder realizar a conversão pretendida, é importante saber que a primeira linha do CSV dado funciona como cabeçalho
que define o que representa cada coluna.
Por exemplo, o seguinte ficheiro "`alunos.csv`":

```
Número,Nome,Curso
3162,Cândido Faísca,Teatro
7777,Cristiano Ronaldo,Desporto
264,Marcelo Sousa,Ciência Política
```
Que corresponde a uma tabela com 3 registos de informação: a primeira linha de cabeçalho identifica os campos de cada registo, `Número`, `Nome`, `Curso`, e as linhas seguintes contêm os registos de informação.

No entanto, os CSV recebidos poderão conter algumas extensões cuja semântica se explica a seguir:

### 1. Listas

No cabeçalho, cada campo poderá ter um número `N` que representará o número de colunas que esse campo abrange.
Por exemplo, imaginemos que ao exemplo anterior se acrescentou um campo `Notas`, com `N = 5` ("`alunos2.csv`"):

```
Número,Nome,Curso,Notas{5},,,,,
3162,Cândido Faísca,Teatro,12,13,14,15,16
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,18
```

Isto significa que o campo Notas abrange 5 colunas (reparem que se colocaram os campos que sobram a vazio, para o
**CSV bater certo**).

### 2. Listas com um intervalo de tamanhos

Para além de um tamanho único, podemos também definir um intervalo de tamanhos `{N, M}`, significando que o número de
colunas de um certo campo pode ir de `N` até `M` ("`alunos3.csv`"):

```
Número,Nome,Curso,Notas{3,5},,,,,
3162,Cândido Faísca,Teatro,12,13,14,,
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,
```

À semelhança do ponto anterior, havendo colunas vazias, os separadores têm de estar lá, o número de colunas deverá ser sempre igual ao valor máximo de colunas, poderão é estar preenchidas com informação ou não.

### 3. Funções de agregação

Para além de listas, podemos ter funções de agregação, aplicadas a essas listas.
Veja os seguintes exemplos: "`alunos4.csv`"

```
Número,Nome,Curso,Notas{3,5}::sum,,,,,
3162,Cândido Faísca,Teatro,12,13,14,,
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,
```

 e "`alunos5.csv`":

 ```
Número,Nome,Curso,Notas{3,5}::media,,,,,
3162,Cândido Faísca,Teatro,12,13,14,,
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,
 ```

## Resultados esperados

O resultado final esperado é um ficheiro JSON resultante da conversão dum ficheiro CSV.
Por exemplo, o ficheiro "`alunos.csv`" (original), deveria ser transformado no seguinte ficheiro "`alunos.json`":

```
[
  {
    "Número": "3612",
    "Nome": "Cândido Faísca",
    "Curso": "Teatro"
  },
  {
    "Número": "7777",
    "Nome": "Cristiano Ronaldo",
    "Curso": "Desporto"
  },
  {
    "Número": "264",
    "Nome": "Marcelo Sousa",
    "Curso": "Ciência Política"
  }
]
```

No caso de existirem listas, os campos que representam essas listas devem ser mapeados para listas em JSON ("`alunos2.csv ==> alunos2.json`"):

```
[
  {
    "Número": "3612",
    "Nome": "Cândido Faísca",
    "Curso": "Teatro",
    "Notas": [12,13,14,15,16]
  },
  {
    "Número": "7777",
    "Nome": "Cristiano Ronaldo",
    "Curso": "Desporto",
    "Notas": [17,12,20,11,12]
  },
  {
    "Número": "264",
    "Nome": "Marcelo Sousa",
    "Curso": "Ciência Política",
    "Notas": [18,19,19,20,18]
  }
]
```

Nos casos em que temos uma lista com uma função de agregação, o processador deve executar a função associada à lista, e
colocar o resultado no JSON, identificando na chave qual foi a função executada ("`alunos4.csv ==> alunos4.json`"):

```
[
  {
    "Número": "3612",
    "Nome": "Cândido Faísca",
    "Curso": "Teatro",
    "Notas_sum": 39
  },
  {
    "Número": "7777",
    "Nome": "Cristiano Ronaldo",
    "Curso": "Desporto",
    "Notas_sum": 72
  },
  {
    "Número": "264",
    "Nome": "Marcelo Sousa",
    "Curso": "Ciência Política",
    "Notas_sum": 76
  }
]
```

In [2]:
import re


def csv2json(file, indent):
    with open(file, 'r') as csv:
        lines = [line.strip().split(',') for line in csv.readlines()]
        header = [h for h in lines[0] if h != '']

        for i in range(len(header)):
            if '}' in header[i] and header[i-1][-1].isdigit():
                header[i-1] += ',' + header[i]
                header.pop(i)
        
        data = []
        for l in lines[1:]:
            a = {}
            for i in range(len(header)):
                rl = re.search(r'{(\d+,)?(\d+)}', header[i])
                rf = re.search(r'{(\d+,)?(\d+)}::(\w+)', header[i])

                if rf != None:
                    h = re.sub(r'{(\d+,)?(\d+)}::(\w+)', r'_\3', header[i])
                    alcance = int(rf.group(2))
                    
                    lista = [int(e) for e in l[i:i+alcance] if e != '']
                    if(rf.group(3) == 'sum'):
                        a[h] = sum(lista)
                    elif(rf.group(3) == 'media'):
                        a[h] = sum(lista) / len(lista)
                elif rl != None:
                    h = re.sub(r'{(\d+,)?\d+}', r'', header[i])
                    alcance = int(rl.group(2))

                    lista = [int(e) for e in l[i:i+alcance] if e != '']
                    a[h] = lista
                else:
                    a[header[i]] = l[i]

            data.append(a)
    csv.close()

    filename = re.sub(r'(.+)(\..*)', r'\1', file)
    with open(filename + '.json', 'w') as write_file:
        out = '[\n'
        for d in data:
            out += indent*' ' + '{\n'
            for k in list(d.keys()):
                if(type(d[k]) != str):
                    d[k] = str(d[k])
                else:
                    d[k] = '"' + d[k] + '"'

                if k!=list(d.keys())[-1]:
                    out += 2*indent*' ' + '"' + k + '": ' + d[k] + ',\n'
                else:
                    out += 2*indent*' ' + '"' + k + '": ' + d[k] + '\n'
            
            if d!=data[-1]:
                out += indent*' ' + '},\n'
            else:
                out += indent*' ' + '}\n'
        out += ']'
        write_file.write(out)

    write_file.close()

for a in ['alunos.csv', 'alunos2.csv', 'alunos3.csv', 'alunos4.csv', 'alunos5.csv']:
    csv2json(a, indent=2)